In [1]:
import pandas as pd
import plotly.express as px

In [2]:
df_principal = pd.read_excel('acoes_pura.xlsx', sheet_name='Principal')

In [3]:
df_total_acoes = pd.read_excel('acoes_pura.xlsx', sheet_name='Total_de_acoes')

df_total_acoes = df_total_acoes.rename(columns={'Código':'ativo'}).copy()

df_total_acoes = df_total_acoes.rename(columns={'Qtde. Teórica':'qtde_teorica'}).copy()

#df_total_acoes['qtde_teorica'] = df_total_acoes['qtde_teorica'].astype(int)

#df_total_acoes.head(10)

In [4]:
#df_total_acoes

In [5]:
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)

#df_total_acoes

In [6]:
df_ticker = pd.read_excel('acoes_pura.xlsx', sheet_name='Ticker')

df_ticker = df_ticker.rename(columns={'Ticker':'ativo','Nome':'nome'}).copy()

#df_ticker.head(10)

In [7]:
df_chatgpt = pd.read_excel('acoes_pura.xlsx', sheet_name='ChatGPT')

df_chatgpt = df_chatgpt.rename(columns={'Nome da empresa':'nome','Segmento':'segmento','Idade (anos)':'idade'}).copy()

#df_chatgpt.head(10)

In [8]:
df_principal = df_principal[['Ativo', 'Data', 'Último (R$)', 'Var. Dia (%)']].copy()

#df_principal.head(10)

In [9]:
df_principal = df_principal.rename(columns={'Último (R$)':'valor_final','Var. Dia (%)':'var_dia_pct'}).copy()

#df_principal

In [10]:
df_principal = df_principal.rename(columns={'Ativo':'ativo','Data':'data'})

#df_principal

In [11]:
df_principal['var_pct'] = df_principal['var_dia_pct'] / 100

#df_principal

In [12]:
df_principal['valor_inicial'] = df_principal['valor_final'] / (1 + df_principal['var_pct'])

#df_principal

In [13]:
df_principal = df_principal.merge(df_total_acoes, left_on='ativo',right_on='ativo',how='left')

#df_principal

In [14]:
#df_principal = df_principal.drop(columns=['Código'])

#Remover coluna do DataFrame

In [37]:
df_principal

,ativo,data,valor_final,var_dia_pct,var_pct,valor_inicial,qtde_teorica,variacao_rs,resultado,nome,segmento,idade,resultado_idade
0,USIM5,2024-01-26,9.50,5.20,0.05,9.03,515117391,241889725.43,Subiu,Usiminas,Siderurgia,60,Entre 50 e 100
1,CMIN3,2024-01-26,6.82,2.40,0.02,6.66,1110559345,177515970.30,Subiu,CSN Mineração,Mineração,8,Abaixo de 50
2,PETR3,2024-01-26,41.96,2.19,0.02,41.06,-2147483648,2140059393.53,Subiu,Petrobras,Petróleo,69,Entre 50 e 100
4,SUZB3,2024-01-26,52.91,2.04,0.02,51.85,683452836,722946282.71,Subiu,Suzano,Papel e Celulose,94,Entre 50 e 100
5,CPFE3,2024-01-26,37.10,2.03,0.02,36.36,187732538,138573955.06,Subiu,CPFL Energia,Energia,109,Acima de 100
...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,CRFB3,2024-01-26,10.71,-2.45,-0.02,10.98,533990587,-143635530.57,Desceu,Carrefour Brasil,Varejo,37,Abaixo de 50
83,BHIA3,2024-01-26,8.70,-2.46,-0.02,8.92,94843047,-20810240.84,Desceu,Casas Bahia,Varejo,95,Entre 50 e 100
84,RENT3,2024-01-26,56.24,-3.63,-0.04,58.36,853202347,-1807432634.46,Desceu,Localiza,Aluguel de Carros,49,Abaixo de 50
85,CVCB3,2024-01-26,3.07,-4.36,-0.04,3.21,525582771,-73557408.06,Desceu,CVC,Turismo,50,Entre 50 e 100


In [16]:
df_principal['variacao_rs'] = (df_principal['valor_final']-df_principal['valor_inicial']) * df_principal['qtde_teorica']

In [17]:
pd.options.display.float_format = '{:.2f}'.format

In [18]:
df_principal['qtde_teorica'] = df_principal['qtde_teorica'].astype(int)

In [19]:
df_principal['resultado'] = df_principal['variacao_rs'].apply(lambda x: 'Subiu' if x > 0 else ('Desceu 'if x < 0 else 'Estável'))

In [20]:
df_principal = df_principal.rename(columns={'Resultado':'resultado'}).copy()

In [21]:
df_principal = df_principal.merge(df_ticker,left_on='ativo',right_on='ativo',how='left')

In [22]:
df_principal = df_principal.merge(df_chatgpt,left_on='nome',right_on='nome',how='left')

In [23]:
df_principal['resultado_idade'] = df_principal['idade'].apply(lambda x: 'Acima de 100' if x > 100 else ('Abaixo de 50' if x < 50 else 'Entre 50 e 100'))

In [24]:
df_principal = df_principal.drop_duplicates('ativo')

In [25]:
#Análises

maior_variacao = df_principal['variacao_rs'].max()

menor_variacao = df_principal['variacao_rs'].min()

media_variacao = df_principal['variacao_rs'].mean()

media_variacao_subiu = df_principal[df_principal['resultado']=='Subiu']['variacao_rs'].mean()

media_variacao_desceu = df_principal[df_principal['resultado']=='Desceu ']['variacao_rs'].mean()

print(f'A Maior variação: R$ {maior_variacao:,.2f}')

print(f'A Menor variação: R$ {menor_variacao:,.2f}')

print(f'A Média de variação: R$ {media_variacao:,.2f}')

print(f'A Média de variação dos que Subiram: R$ {media_variacao_subiu:,.2f}')

print(f'A Média de variação dos que Desceram: R$ {media_variacao_desceu:,.2f}')


A Maior variação: R$ 4,762,926,995.25
A Menor variação: R$ -1,807,432,634.46
A Média de variação: R$ 165,190,210.48
A Média de variação dos que Subiram: R$ 448,164,250.23
A Média de variação dos que Desceram: R$ -181,109,141.75


In [26]:
df_principal_subiu = df_principal[df_principal['resultado']=='Subiu']

df_principal_subiu = df_principal_subiu.drop(columns='resultado')

In [27]:
#df_principal_subiu

In [28]:
df_principal_desceu = df_principal[df_principal['resultado']=='Desceu ']

df_principal_desceu = df_principal_desceu.drop(columns='resultado')

In [29]:
#df_principal_desceu

In [30]:
df_analise_segmento = df_principal_subiu.groupby('segmento')['variacao_rs'].sum().reset_index()

#df_analise_segmento

In [31]:
df_analise_saldo = df_principal.groupby('resultado')['variacao_rs'].sum().reset_index()

#df_analise_saldo

In [32]:
fig_analise_saldo = px.bar(df_analise_saldo, x='resultado' ,y='variacao_rs',text='variacao_rs', title='Variação Resultado por R$')

fig_analise_saldo.update_traces(texttemplate='%{text:,.2f}', textposition='inside')

fig_analise_saldo.update_layout(width=800, height=500)

fig_analise_saldo.show()

In [33]:
df_analise_idade = df_principal_subiu.groupby('resultado_idade')['variacao_rs'].sum().reset_index()

df_analise_idade

,resultado_idade,variacao_rs
0,Abaixo de 50,5485660198.58
1,Acima de 100,992503730.82
2,Entre 50 e 100,13241063080.74


In [34]:
fig_analise_idade = px.bar(df_analise_idade, x='resultado_idade',y='variacao_rs',text='variacao_rs',title='Variação Resultado por Idade')

fig_analise_idade.update_traces(texttemplate='%{text:,.2f}', textposition='inside')

fig_analise_idade.update_layout(width=800, height=500)

fig_analise_idade.show()

In [35]:
df_analise_segmento_subiu = df_principal[df_principal['resultado']=='Subiu'].groupby('segmento')['variacao_rs'].sum().reset_index()

#df_analise_segmento_subiu

In [36]:
fig_analise_segmento_subiu = px.pie(df_analise_segmento_subiu, names='segmento',values='variacao_rs',title='Análise de Segmento')

fig_analise_segmento_subiu.update_layout(width=600, height=500)

fig_analise_segmento_subiu.show()